Matrix free operations
====

* By proper mapping, differential operators become geometry-free
* For $L_2$-like spaces mass matrices become diagonal, and for 
vectorial $L_2$ $d\times d$-block diagonal
* for variable coefficient / curved elements we apply sum-factorization for $L_2$-spaces.
* compose second order operators by operator algebra
* simple preconditioning 

In [ ]:
from netgen.geom2d import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.comp import ConvertOperator
from ngsolve.krylovspace import CGSolver
from time import time

In [ ]:
geo = CSG2d()
circle = Circle( center=(0,0), radius=1.0, \
                mat="mat1", bc="outer" )
geo.Add (circle)

mesh = Mesh(geo.GenerateMesh(maxh=0.5))
mesh.Curve(3)
Draw (mesh);

The Gradient Operator from $H^1$ to $[L_2]^2$
---

We can generate the matrix representation for the Gradient operator as a sparse matrix

In [ ]:
order = 3
fes = H1(mesh, order=order)
u,v = fes.TnT()

fesgrad = VectorL2(mesh, order=order-1)
gradop1 = ConvertOperator(fes, fesgrad, grad(u))

print (gradop1)

In [ ]:
gfu = GridFunction(fes)
gfu.Set (x*y)
gfgrad = GridFunction(fesgrad)
gfgrad.vec.data = gradop1 * gfu.vec
Draw (gfgrad[0], mesh, order=3);

When we apply covariant mappings for the $[L_2]^2$-space, the gradient operator becomes independent of geometry:

In [ ]:
fesgrad2 = VectorL2(mesh, order=order-1, covariant=True)
gradop2 = ConvertOperator(fes, fesgrad2, grad(u))
print (gradop2)

In [ ]:
gfgrad2 = GridFunction(fesgrad2)
gfgrad2.vec.data = gradop2 * gfu.vec
Draw (gfgrad2[0], mesh, order=3);

This can be utilized by setting the flag `geom_free` for the convert operator. Then only the gradient for the reference element (or serveral of them) is computed. We deal with the so called sign conflict by sorting the elements into equivalence classes (need 2 for triangular meshes).

In [ ]:
fesgrad3 = VectorL2(mesh, order=order-1, covariant=True)
gradop3 = ConvertOperator(fes, fesgrad3, grad(u), geom_free=True)
print (gradop3)    # not a sparse matrix, but does not tell much

In [ ]:
gfgrad3 = GridFunction(fesgrad3)
gfgrad3.vec.data = gradop3 * gfu.vec
Draw (gfgrad3[0], mesh, order=3);

Solving the Poisson equation with matrix-free operators
---

In [ ]:
mesh = Mesh(geo.GenerateMesh(maxh=0.2))
for l in range(2):
     mesh.ngmesh.Refine()
mesh.Curve(5)
Draw (mesh);

In [ ]:
order=10
fes = H1(mesh, order=order, dirichlet="outer")
print ("ndof =", fes.ndof)
u,v = fes.TnT()

fesgrad = VectorL2(mesh, order=order-1, covariant=True)
gradop = ConvertOperator(fes, fesgrad, grad(u), geom_free=True)
material = fesgrad.Mass(1)    # mass matrix operator

A composition of linear operators:

In [ ]:
laplaceop = gradop.T @ material @ gradop

We apply diagonal preconditioning:

In [ ]:
adiag = BilinearForm(grad(u)*grad(v)*dx, diagonal=True).Assemble()
invdiag = adiag.mat.Inverse(freedofs=fes.FreeDofs())

In [ ]:
inv = CGSolver(laplaceop, invdiag, printrates='\r', maxiter=2000)

In [ ]:
f = LinearForm(x*v*dx).Assemble()
gfu = GridFunction(fes)

In [ ]:
from time import time
ts = time()
with TaskManager():
    gfu.vec.data = inv * f.vec
te = time()
print ("Time: ", te-ts)

In [ ]:
Draw (gfu);

Low order space:
---

In [ ]:
feslo = H1(mesh, order=1, dirichlet="outer")
ulo,vlo = feslo.TnT()
alo = BilinearForm(grad(ulo)*grad(vlo)*dx).Assemble()
ainvlo1 = alo.mat.Inverse(feslo.FreeDofs(), inverse="sparsecholesky")
emb = Embedding(fes.ndof, IntRange(0, feslo.ndof))
ainvlo = emb @ ainvlo1 @ emb.T

In [ ]:
pre = ainvlo + invdiag
with TaskManager():
    inv = CGSolver(laplaceop, pre, printrates='\r', maxiter=2000)
gfu.vec.data = inv * f.vec

In [ ]:
Draw (gfu, order=3);